# Reactiva Perú – Reprogramación de Créditos
## Unidad 1: Perfil del Proyecto y Dataset Preprocesado (CRISP-DM)

**Autores:** [Agrega tus nombres]

**Curso:** Minería de Datos

**Fecha:** 2025-09-08

---
Este cuaderno implementa la primera entrega del proyecto siguiendo **CRISP-DM**:
1) Comprensión del negocio.
2) Comprensión y preparación de los datos.
3) Planificación del modelado.
4) Entrega de dataset limpio y documentado.


## 1. Perfil del Proyecto

### 1.1 Título
Análisis y predicción de la reprogramación de créditos de Reactiva Perú mediante técnicas de minería de datos

### 1.2 Problema / Caso de negocio
El programa Reactiva Perú otorgó créditos con garantía estatal a empresas para sostener su capital de trabajo durante la pandemia. Una proporción relevante de empresas reprogramó sus créditos. Interesa identificar factores asociados a la reprogramación y preparar un dataset que permita, en fases posteriores, construir modelos predictivos.

### 1.3 Propósito / Valor esperado
- Detectar patrones asociados a la reprogramación de créditos.
- Entregar un dataset limpio y estructurado apto para modelado.
- Sentar bases para modelos de clasificación en fases siguientes.

### 1.4 Justificación
- Evaluar la eficacia del programa frente a la crisis.
- Reconocer sectores/regiones con mayor vulnerabilidad.
- Aplicar técnicas de minería de datos en un caso real con valor público.

### 1.5 Objetivos
**General:** Analizar y preparar datos de Reactiva Perú para explicar y predecir reprogramación de créditos.

**Específicos:**

1. Describir el contexto y variables del dataset.

2. Aplicar limpieza, transformación y documentación del dataset (ETL/ELT).

3. Planificar técnicas de modelado (clasificación) y métricas de evaluación.


### 1.6 Alcance
Se trabaja con datos públicos del MEF sobre reprogramación de créditos. No se usa información financiera privada ni se realiza análisis macroeconómico completo.


## 2. Metodología: CRISP-DM

1) **Comprensión del negocio:** definición del problema y objetivos.
2) **Comprensión de los datos:** inventario de variables, calidad de datos, distribución.
3) **Preparación de datos:** limpieza, transformación, codificación, reducción.
4) **Modelado (planificación):** propuesta de algoritmos y métricas.
5) **Evaluación (planificación):** criterios: precisión, recall, F1.
6) **Despliegue (futuro):** uso para decisiones públicas/financieras.


In [ ]:
# 3. Configuración e importaciones
import os
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 160)

print('Versiones -> pandas:', pd.__version__, '| numpy:', np.__version__)


In [ ]:
# 4. Ruta de datos y carga
# Ajusta la ruta y el nombre de archivo si corresponde.
# Se soporta un único Excel o una lista de archivos para concatenación.

RUTA_DATOS = './'  # cambia si el archivo está en otra carpeta
ARCHIVOS_EXCEL = [
    'reactiva_peru_2022.xlsx'  # ejemplo: reemplaza por el nombre real si es distinto
]

# Lectura y concatenación
frames = []
for f in ARCHIVOS_EXCEL:
    path = os.path.join(RUTA_DATOS, f)
    if not os.path.exists(path):
        print('Advertencia: no se encontró', path)
    else:
        frames.append(pd.read_excel(path))

if frames:
    df_raw = pd.concat(frames, ignore_index=True)
else:
    # fallback: crear un df vacío para no romper el cuaderno
    df_raw = pd.DataFrame()

print('Forma del dataset original:', df_raw.shape)
df_raw.head(3)


## 5. Comprensión de los Datos – Diccionario inicial

Documentar variables clave. Completar el CSV `diccionario_variables.csv` generado al final de este cuaderno.

Variables típicas en Reactiva Perú (ajustar según columnas reales):

- ORDEN: índice interno del registro.

- RAZÓN SOCIAL: nombre de la empresa (identificador, no se usa para modelado).

- RUC/DNI: identificador numérico de la empresa (no para modelado).

- SECTOR ECONÓMICO: rubro de la empresa.

- DEPARTAMENTO: ubicación geográfica.

- ENTIDAD FINANCIERA: banco o financiera otorgante.

- SALDO INSOLUTO: monto pendiente de pago.

- COBERTURA SALDO INSOLUTO: monto garantizado por el Estado.

- REPROGRAMACIÓN: indicador de si el crédito fue reprogramado (Sí/No).



In [ ]:
# 6. Auditoría de calidad de datos
print('Columnas:', list(df_raw.columns))
print('\nTipos de datos:')
print(df_raw.dtypes)

print('\nValores nulos por columna:')
print(df_raw.isna().sum())

# Duplicados (considera todas las columnas para detectar duplicados idénticos)
duplicados = df_raw.duplicated().sum() if not df_raw.empty else 0
print('\nDuplicados totales:', duplicados)

# Estadísticos básicos para numéricas
num_cols = df_raw.select_dtypes(include=[np.number]).columns.tolist()
if num_cols:
    display(df_raw[num_cols].describe())
else:
    print('No hay columnas numéricas detectadas.')


In [ ]:
# 7. EDA básica (sin librerías externas, solo matplotlib)

# Distribución de una variable numérica clave (ajustar nombre si difiere)
col_monto = None
for c in df_raw.columns:
    if 'SALDO' in c.upper() or 'MONTO' in c.upper():
        col_monto = c
        break

if col_monto is not None:
    plt.figure(figsize=(7,4))
    df_raw[col_monto].dropna().astype(float).plot(kind='hist', bins=40)
    plt.title(f'Distribución de {col_monto}')
    plt.xlabel(col_monto)
    plt.ylabel('Frecuencia')
    plt.show()
else:
    print('No se encontró columna de monto o saldo para histograma.')

# Conteo por sector económico (o variable categórica relevante)
cat_col = None
candidates = ['SECTOR', 'SECTOR ECONÓMICO', 'SECTOR_ECONOMICO']
upper_cols = {c.upper(): c for c in df_raw.columns}
for cand in candidates:
    if cand in upper_cols:
        cat_col = upper_cols[cand]
        break

if cat_col is not None:
    conteo = df_raw[cat_col].astype(str).value_counts().head(15)
    plt.figure(figsize=(8,4))
    conteo.plot(kind='bar')
    plt.title(f'Top 15 categorías en {cat_col}')
    plt.xlabel(cat_col)
    plt.ylabel('Frecuencia')
    plt.tight_layout()
    plt.show()
else:
    print('No se encontró columna de sector económico para gráfico de barras.')


In [ ]:
# 8. Preparación de datos: limpieza

df = df_raw.copy()

# Estandarización de nombres de columnas
df.columns = [str(c).strip().replace('\n',' ').replace('  ',' ') for c in df.columns]

# Eliminación de duplicados exactos
df = df.drop_duplicates().reset_index(drop=True)

# Conversión de posibles montos a numérico (intenta detectar columnas con 'SALDO' o 'MONTO')
for c in df.columns:
    if any(k in c.upper() for k in ['SALDO', 'MONTO']):
        # Quitar comas, espacios y símbolos si existen, luego convertir
        df[c] = (
            df[c]
            .astype(str)
            .str.replace('[^0-9\.-]', '', regex=True)
            .replace({'': np.nan, '-': np.nan})
        )
        df[c] = pd.to_numeric(df[c], errors='coerce')

# Tratamiento de nulos básicos: ejemplo simple (ajustar a criterio)
# Regla: si una columna numérica tiene pocos nulos, rellenar con mediana; si muchos nulos, evaluar descartar
threshold = 0.3  # 30% tolerancia
for c in df.columns:
    null_rate = df[c].isna().mean()
    if df[c].dtype.kind in 'biufc':  # numérica
        if null_rate > 0 and null_rate <= threshold:
            df[c] = df[c].fillna(df[c].median())
    else:
        # Categóricas: rellenar nulos con 'DESCONOCIDO' si es bajo el umbral
        if null_rate > 0 and null_rate <= threshold:
            df[c] = df[c].fillna('DESCONOCIDO')

# Eliminación de columnas con demasiados nulos
cols_to_drop = [c for c in df.columns if df[c].isna().mean() > 0.6]
df = df.drop(columns=cols_to_drop)

print('Columnas eliminadas por nulos > 60%:', cols_to_drop)
print('Forma tras limpieza:', df.shape)
df.head(3)


In [ ]:
# 9. Preparación de datos: selección y codificación

# Identificadores a descartar del modelado
id_like = []
for c in df.columns:
    cu = c.upper()
    if any(k in cu for k in ['RUC', 'DNI', 'RAZON', 'RAZÓN', 'ORDEN', 'NOMBRE']):
        id_like.append(c)

df_model = df.drop(columns=id_like, errors='ignore')

# Variable objetivo: REPROGRAMACIÓN (ajustar a nombre real)
y_col = None
for c in df.columns:
    if 'REPRO' in c.upper():
        y_col = c
        break

if y_col is not None:
    # Convertir a binaria 1/0 si es texto
    if df_model[y_col].dtype == object:
        df_model[y_col] = df_model[y_col].str.strip().str.upper().map({'SI':1, 'SÍ':1, 'YES':1, 'Y':1, '1':1, 'NO':0, 'N':0, '0':0})
    # Asegurar tipo numérico
    df_model[y_col] = pd.to_numeric(df_model[y_col], errors='coerce')

# One-Hot Encoding para categóricas
cat_cols = df_model.select_dtypes(include=['object']).columns.tolist()
df_model = pd.get_dummies(df_model, columns=cat_cols, dummy_na=False)

print('Forma dataset de modelado (temporal):', df_model.shape)
df_model.head(3)


In [ ]:
# 10. Normalización opcional de variables numéricas (no se aplica a la variable objetivo)

from sklearn.preprocessing import MinMaxScaler

if 'y_col' in locals() and y_col in df_model.columns:
    features = [c for c in df_model.columns if c != y_col]
else:
    features = df_model.columns.tolist()

scaler = MinMaxScaler()
df_scaled = df_model.copy()
df_scaled[features] = scaler.fit_transform(df_model[features])

print('Forma del dataset escalado:', df_scaled.shape)
df_scaled.head(3)


In [ ]:
# 11. Guardado de artefactos
#  - Dataset limpio (df)
#  - Dataset modelado (df_model)
#  - Dataset escalado (df_scaled)
#  - Diccionario de variables (plantilla)

OUTPUT_DIR = './salidas_unidad1'
os.makedirs(OUTPUT_DIR, exist_ok=True)

path_clean = os.path.join(OUTPUT_DIR, 'reactiva_clean.csv')
path_model = os.path.join(OUTPUT_DIR, 'reactiva_model.csv')
path_scaled = os.path.join(OUTPUT_DIR, 'reactiva_scaled.csv')

df.to_csv(path_clean, index=False, encoding='utf-8')
df_model.to_csv(path_model, index=False, encoding='utf-8')
df_scaled.to_csv(path_scaled, index=False, encoding='utf-8')

print('Guardado:')
print(' -', path_clean)
print(' -', path_model)
print(' -', path_scaled)


In [ ]:
# 12. Plantilla de diccionario de variables y tratamientos
cols = list(df_raw.columns)
template = pd.DataFrame({
    'variable_original': cols,
    'descripcion': [''] * len(cols),
    'tipo_dato': ['numerica/categorica/fecha'] * len(cols),
    'tratamientos_aplicados': ['limpieza/codificacion/normalizacion/otra'] * len(cols),
    'observaciones': [''] * len(cols)
})
DICT_PATH = os.path.join(OUTPUT_DIR, 'diccionario_variables.csv')
template.to_csv(DICT_PATH, index=False, encoding='utf-8')
print('Plantilla creada:', DICT_PATH)
template.head(10)


## 13. Planificación para el Modelado

Problema de clasificación binaria: predecir la reprogramación del crédito.
Modelos candidatos: Regresión Logística, Árboles de Decisión, Random Forest.
Métricas: precisión, recall, F1-score. Validación con train/test split o validación cruzada.


In [ ]:
# 13.1 División de datos (borrador para próxima unidad)
from sklearn.model_selection import train_test_split

if 'y_col' in locals() and y_col in df_scaled.columns:
    X = df_scaled.drop(columns=[y_col])
    y = df_scaled[y_col].fillna(0).astype(int)  # fallback simple
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
else:
    print('No se detectó variable objetivo de reprogramación. Define la columna que indica REPRO.')


## 14. Conclusiones iniciales

1) Se documentó el contexto y los objetivos del proyecto.
2) Se auditó la calidad de datos y se aplicaron reglas básicas de limpieza.
3) Se generó un dataset limpio y versiones transformadas aptas para modelado.
4) Se dejó preparado el pipeline para partición de datos y futura etapa de modelado.
